<h1> Introduction </h1>

The objective of this data analysis is to gain valuable insights into the current job market for Software Engineers and Web Developers. The data was collected by web scraping from the Indeed website on July 21st, 2023. The analysis includes information such as job titles, company details, locations, job types, salary ranges, and the specific skills required for each job posting. The search terms used for web scraping include "Software Engineer," "Software Developer," and "Web Developer." Additionally, the results were filtered to include only postings that were posted within the last 14 days to ensure the data reflects current job opportunities.

<h3> Installation </h3>

In [1]:
!pip install spacy
!pip install --upgrade pip

<h3> Import Statements </h3>

In [2]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

/Users/meihuang/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h3> Notebook Presentation </h3>

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

<h3> Reading data </h3>

In [4]:
df_data = pd.read_csv('software_engineer_jobs_USA.csv',sep=';')

<h3> Data Exploration </h3>

In [5]:
print(f'(Rows,Columns): {df_data.shape}')

(Rows,Columns): (3556, 5)


In [6]:
df_data.tail(5)

,job_title,company_name,company_location,salary,skills
3551,Software Developer,"CSSI, Inc.","Washington, DC 20024 \n(Southwest Waterfront a...",NaN,"['JavaScript', 'Java', 'C#', 'Go', 'SQL', 'CSS..."
3552,Software Engineer,Calfus,"Pleasanton, CA",NaN,"['Java', 'Go', 'SQL', 'NoSQL', 'React', 'Node...."
3553,Python Developer,Gatix E Solution,"Texas City, TX",$30 - $60 an hour,"['Python', 'JavaScript', 'Java', 'Go', 'SQL', ..."
3554,Associate Software Engineer,McKesson,"Atlanta, GA 30329",$38.08 - $63.46 an hour,"['JavaScript', 'Java', 'C#', 'Go', 'TypeScript..."
3555,Software Engineer,"X-Force, LLC.","Fort Leavenworth, KS 66027",$50 an hour,"['JavaScript', 'Java', 'C#', 'SQL', 'HTML', 'A..."


In [7]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3556 entries, 0 to 3555
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         3556 non-null   object
 1   company_name      3556 non-null   object
 2   company_location  3556 non-null   object
 3   salary            1964 non-null   object
 4   skills            3556 non-null   object
dtypes: object(5)
memory usage: 139.0+ KB


<h3> Check for Nan Values and Duplicates </h3>

In [8]:
print(f'Missing values?: {df_data.isna().values.any()}')
df_data.isna().sum()

Missing values?: True


job_title              0
company_name           0
company_location       0
salary              1592
skills                 0
dtype: int64

In [9]:
print(f'Duplicate values?: {df_data.duplicated().values.any()}')
df_data.duplicated().sum()

Duplicate values?: True


1979

<h2> Data Cleaning </h2> 

<h4> Drop data with empty values </h4>

In [10]:
df_data_clean = df_data.dropna()
df_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 3555
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         1964 non-null   object
 1   company_name      1964 non-null   object
 2   company_location  1964 non-null   object
 3   salary            1964 non-null   object
 4   skills            1964 non-null   object
dtypes: object(5)
memory usage: 92.1+ KB


<h4> Drop duplicate data with the same company name and job title </h4>

In [11]:
df_data_clean.drop_duplicates(subset=['company_name', 'job_title', 'company_location'],inplace=True)
df_data_clean.shape

/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/1019937765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean.drop_duplicates(subset=['company_name', 'job_title', 'company_location'],inplace=True)


(842, 5)

In [12]:
df_data_clean.sample(5)

,job_title,company_name,company_location,salary,skills
2033,REMOTE- Senior Front End Developer (React),CyberCoders,"Remote in Needham, MA 02494","$140,000 - $170,000 a year","['TypeScript', 'React']"
1903,Automation Engineer / Mechatronics,HRC,"Bremen, IN","$90,000 - $130,000 a year","['Go', 'Git', 'Foundation']"
673,"Mid-Level Software Engineer, .NET Full Stack",ONEflight International,"Englewood, CO 80112","$95,000 - $115,000 a year","['C#', 'SQL', 'HTML', 'CSS', 'ASP.NET', 'Git',..."
327,"Software Engineer, Alternate Content Engineering",Disney,"Santa Monica, CA 90401 \n(Downtown/Third Stree...","$112,586 - $151,030 a year","['Python', 'Java', 'Go', 'SQL', 'Django', 'Spr..."
1334,Principal Frontend Software Engineer,Motion Recruitment,"Raleigh, NC 27513","$150,000 - $180,000 a year","['JavaScript', 'Java', 'React', 'AWS']"


<h3> Extract minimum salary for each job </h3>

In [13]:
# Create a new column called salary min 
df_data_clean['salary_min'] = df_data_clean['salary'].str.split('-').str[0]
df_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 3553
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         842 non-null    object
 1   company_name      842 non-null    object
 2   company_location  842 non-null    object
 3   salary            842 non-null    object
 4   skills            842 non-null    object
 5   salary_min        842 non-null    object
dtypes: object(6)
memory usage: 46.0+ KB


/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/4110173538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean['salary_min'] = df_data_clean['salary'].str.split('-').str[0]


In [14]:
df_data_clean.sample(5)

,job_title,company_name,company_location,salary,skills,salary_min
1484,Software Developer II (Web-Mobile) - Remote,Navitus Health Solutions LLC,Remote,"$75,287 - $101,615 a year","['JavaScript', 'Java', 'C#', 'SQL', 'HTML', 'C...","$75,287"
156,Angular Developer,Matlen Silver,"Charlotte, NC 28202 \n(Downtown Charlotte area)",$65 - $70 an hour,"['JavaScript', 'Java', 'Go', 'HTML', 'CSS', 'A...",$65
2515,IT Systems Developer/Specialist,Graham Personnel Services,"Marshville, NC","$70,000 - $75,000 a year",['C#'],"$70,000"
1024,Windows Application Developer,Earthley.com,"Columbus, OH 43213 \n(East Broad area)",$30 - $50 an hour,"['C++', 'C#', 'Go', 'SQL', 'Git', 'AWS', 'Wind...",$30
659,Remote Angular Developer,Vaco,Remote,$55 an hour,"['JavaScript', 'Java', 'TypeScript', 'SQL', 'H...",$55 an hour


In [15]:
def extractNum(text):
    firstchar = str(text).split(" ")[0]
    if firstchar == "From":
        return text.split(" ")[1]
    elif firstchar == "Up":
        return text.split(" ")[2]
    else:
        return firstchar 

In [16]:
df_data_clean.sample(5)

,job_title,company_name,company_location,salary,skills,salary_min
2480,Java Full Stack Developer,XEBIA,"Atlanta, GA 30326 \n(Lenox area)",$65 an hour,"['Python', 'JavaScript', 'Java', 'Swift', 'Go'...",$65 an hour
10,Cloud Software Engineer,Brooksource,"Houston, TX",$50 - $75 an hour,"['Java', 'Go', 'AWS', 'Azure', 'Linux']",$50
168,Software Developers,Volto Consulting,"Schaumburg, IL","$134,118 a year","['Java', 'SQL', 'Jenkins', 'JIRA']","$134,118 a year"
1979,Software Engineer - Engineering Services,Entegee,"Andover, MA","$90,000 - $100,000 a year","['Python', 'JavaScript', 'Java', 'C++', 'C#', ...","$90,000"
1857,Junior Delphi Developer,CDE Software,"Hybrid remote in Tukwila, WA 98168","$47,941 - $50,700 a year","['C#', 'Go', 'SQL', 'Git']","$47,941"


In [17]:
df_data_clean['salary_min'] = df_data_clean['salary_min'].apply(extractNum)
df_data_clean.sample(50)

/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/86085078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean['salary_min'] = df_data_clean['salary_min'].apply(extractNum)


,job_title,company_name,company_location,salary,skills,salary_min
1108,Software Engineering Manager,Nation Safe Drivers,"Boca Raton, FL 33487","$140,000 - $160,000 a year","['Go', 'SQL', 'NoSQL', 'React', 'Angular', 'No...","$140,000"
442,Senior Software Engineer,Liberty Mutual,"Wausau, WI 54401","$100,200 - $185,300 a year","['Java', 'XML', 'Git', 'AWS']","$100,200"
1122,Software Test Engineer,"MPIRE Technology Group, Inc.",Remote,"$80,000 a year","['Python', 'JavaScript', 'Java', 'PHP', 'SQL',...","$80,000"
2103,C#.Net Backend Developer (with SQL),Confidential,"Scottsdale, AZ 85251 \n(South Scottsdale area)",$47 - $75 an hour,"['C#', 'SQL', 'ASP.NET', 'JSON', 'XML', 'Azure']",$47
851,Software Engineer,Montana State University,"Bozeman, MT",$34.88 an hour,"['JavaScript', 'Java', 'Go', 'SQL', 'HTML', 'A...",$34.88
3437,"Senior Automation Engineer, Pharmaceutical Man...","New Age Software Services, Inc","Hybrid remote in Bedford, MA 01730","$120,000 - $130,000 a year",['Go'],"$120,000"
1880,FMS Software Engineer,Atlantis IT Group,"Cedar Rapids, IA","$100,000 - $110,000 a year","['Java', 'JIRA']","$100,000"
1566,Senior Software Engineer,Createch,"Mission, KS 66202",$60 - $70 an hour,"['C#', 'React', 'Angular', 'Vue.js', 'Git', 'A...",$60
2397,Senior WordPress Web Developer,Act! LLC,"Scottsdale, AZ 85258 \n(North Scottsdale area)","$75,000 a year","['JavaScript', 'Java', 'C#', 'PHP', 'Go', 'SQL...","$75,000"
35,Back End Developer,Jacoby and Meyers,"Remote in Los Angeles, CA 90079","$90,000 - $140,000 a year","['Go', 'SQL', 'JSON', 'XML', 'Windows']","$90,000"


In [18]:
## Extract just the numeric portion of the string
def extractMoney(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'MONEY':
            return ent.text
    return None

In [19]:
df_data_clean['salary_min'] = df_data_clean['salary_min'].apply(extractMoney)

/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/3753812553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean['salary_min'] = df_data_clean['salary_min'].apply(extractMoney)


In [20]:
df_data_clean.sample(10)

,job_title,company_name,company_location,salary,skills,salary_min
1371,Full Stack Senior Software Engineer,Motion Recruitment,"Remote in Austin, TX 78701","$150,000 - $180,000 a year","['TypeScript', 'React', 'Git', 'AWS']","150,000"
1929,Staff Software Engineer II (Developer Platform),Coupang,"Seattle, WA 98101 \n(Denny Triangle area)","From $142,000 a year","['Python', 'Java', 'C++', 'Go', 'JSON', 'AWS',...","142,000"
2197,Software Developer,ROOF,"Sarasota, FL 34236",From $22 an hour,"['JavaScript', 'Java', 'PHP', 'Go', 'SQL', 'HT...",22
276,Junior Software Engineer,Alexander Technology Group,"Woburn, MA 01801",$30 an hour,"['Python', 'SQL', 'NoSQL', 'React', 'Angular',...",30
3069,Java Developer,Peraton,"Colorado Springs, CO 80903 \n(Central Colorado...","$66,000 - $106,000 a year","['JavaScript', 'Java', 'Go', 'HTML', 'CSS', 'A...","66,000"
328,Full Stack Software Engineer- AI Climate Tech ...,Somosotech IT Co Limited,"San Francisco, CA 94103 \n(Mission area)","$120,000 - $200,000 a year",[],"120,000"
934,Software Engineer II,VirginPulse,"Remote in Providence, RI 02860","Up to $95,000 a year","['JavaScript', 'Java', 'Swift', 'Go', 'TypeScr...","95,000"
2036,Web Developer,KesarWeb,"Remote in Washington, DC","$85,000 - $94,000 a year","['JavaScript', 'Java', 'PHP', 'Ruby', 'HTML', ...","85,000"
3387,Java Developer,The Tek Studio,"Phoenix, AZ 85003 \n(Central City area)",$50 - $55 an hour,"['JavaScript', 'Java', 'Go', 'SQL', 'HTML', 'C...",50
5,PHP Developer,Peter Glenn Ski & Sports,"Oakland Park, FL 33311 \n(Oakland Forest area)","$59,000 - $80,000 a year","['JavaScript', 'Java', 'PHP', 'Go', 'SQL', 'HT...","59,000"


In [21]:
df_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 3553
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         842 non-null    object
 1   company_name      842 non-null    object
 2   company_location  842 non-null    object
 3   salary            842 non-null    object
 4   skills            842 non-null    object
 5   salary_min        842 non-null    object
dtypes: object(6)
memory usage: 46.0+ KB


In [22]:
## for the hourly salary: multiply those by 2080 which is the average number of working hours 
def findYearly(rate):
    if rate <= 200:
        return rate * 2080
    elif rate >= 1000 and rate <= 10000:
        return rate * 12
    return rate

In [23]:
#remove commas 
df_data_clean['salary_min'] = df_data_clean['salary_min'].str.replace(',','')

/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/628019423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean['salary_min'] = df_data_clean['salary_min'].str.replace(',','')


In [24]:
#convert to float
df_data_clean['salary_min'] = pd.to_numeric(df_data_clean['salary_min'])

/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/2933546722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean['salary_min'] = pd.to_numeric(df_data_clean['salary_min'])


In [25]:
#multiply by 2080 if salary is stated as hourly 
df_data_clean['salary_min'] = df_data_clean['salary_min'].apply(findYearly)
df_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 3553
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_title         842 non-null    object 
 1   company_name      842 non-null    object 
 2   company_location  842 non-null    object 
 3   salary            842 non-null    object 
 4   skills            842 non-null    object 
 5   salary_min        842 non-null    float64
dtypes: float64(1), object(5)
memory usage: 46.0+ KB


/var/folders/wx/n3nghw29261195fbm316p8xm0000gn/T/ipykernel_18354/1306016826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_clean['salary_min'] = df_data_clean['salary_min'].apply(findYearly)


In [26]:
df_data_clean.sample(25)

,job_title,company_name,company_location,salary,skills,salary_min
1072,Contract Software Testing Engineer 1st shift $...,Abundant Solutions,"Allen, TX 75013",$43 an hour,"['Go', 'Linux', 'Windows']","89,440.00"
113,Senior .Net Developer,"Brave New Markets, Inc.","Owings Mills, MD 21117","From $130,000 a year","['JavaScript', 'Java', 'SQL', 'HTML', 'CSS', '...","130,000.00"
16,Principal Cloud Engineer,LPL Financial,"Fort Mill, SC 29715","$149,320 - $223,980 a year","['Python', 'Java', 'C#', 'SQL', 'Docker', 'AWS...","149,320.00"
3104,Founding Software Engineer,Pear VC,"New York, NY 10044 \n(Roosevelt Island area)","$80,000 - $120,000 a year","['Python', 'JavaScript', 'Java', 'SQL', 'React...","80,000.00"
147,Application Analyst/Programmer,Powtec,Remote,"$85,000 - $105,000 a year","['JavaScript', 'Java', 'Go', 'SQL', 'HTML', 'A...","85,000.00"
1358,Lead Front-End Developer (React),Vettafi,Remote,"$140,000 - $160,000 a year","['JavaScript', 'Java', 'Go', 'React', 'Git', '...","140,000.00"
286,Software Engineer (P),DCI Solutions,"Washington, DC","$140,000 - $180,000 a year","['Python', 'Java', 'TypeScript', 'Linux']","140,000.00"
1105,"Software Engineer II, Cloud Engineering","The Travelers Companies, Inc.","Hybrid remote in Hartford, CT 06101","$110,500 - $182,300 a year","['Python', 'Go', 'SQL', 'NoSQL', 'Angular', 'N...","110,500.00"
3174,C# Developer/Senior Engineer,Atlantis IT Group,"Miami, FL",$50 - $60 an hour,"['Java', 'C#', 'Go']","104,000.00"
495,Software Development Engineer in Test *Open to...,Medidata Solutions,"Remote in New York, NY 10003","$95,000 - $125,000 a year","['Python', 'JavaScript', 'Java', 'C#', 'Ruby',...","95,000.00"


In [27]:
df_data_clean.describe()

,salary_min
count,842.00
mean,"104,065.58"
std,"37,532.97"
min,"14,519.00"
25%,"79,010.00"
50%,"100,000.00"
75%,"125,000.00"
max,"350,000.00"


In [29]:
df_data_clean[df_data_clean['salary_min'] == 14519]

,job_title,company_name,company_location,salary,skills,salary_min
1178,React Developer,Steinn Labs LLP,Remote,"$14,519 - $15,000 a year","['JavaScript', 'Java', 'Go', 'HTML', 'CSS', 'R...","14,519.00"
